# RNN
## Recurrent Neural Network　再帰型ニューラルネットワーク



In [1]:
import tensorflow as tf
import input_data
import numpy as np
import time
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
np.set_printoptions(suppress=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Network Parameters

n_classes = 10
n_hidden = 128
sequence_size = 28
frame_size = 28
train_dropout = 0.8
test_dropout = 1.0

# Training Parameters
learning_rate = 0.001
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
print(total_batch)
training_epochs = 30

# tf Graph input
X = tf.placeholder(tf.float32, [None, sequence_size * frame_size])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Store layers weight & bias
weights = {'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))}
biases = {'out': tf.Variable(tf.random_normal([n_classes]))}

550


In [3]:
# Create model


def RNN(x, weights, biases, dropout):

    x = tf.reshape(x, shape=[-1, sequence_size, frame_size])

    #     cell = tf.contrib.rnn.BasicRNNCell(n_hidden)
    #     cell = tf.contrib.rnn.GRUCell(n_hidden)
    cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    cell = tf.contrib.rnn.DropoutWrapper(cell, dropout)

    outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)

    out = tf.add(tf.matmul(outputs[:, -1, :], weights['out']), biases['out'])
    return out

In [4]:
# Construct model
y_pred = RNN(X, weights, biases, keep_prob)
y_softmax = tf.nn.softmax(y_pred)

# Define loss and optimizer
# type 1:
# loss = tf.reduce_mean(
#     -tf.reduce_sum(Y * tf.log(y_softmax), reduction_indices=[1]))
# type 2:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_pred))
# type 3:
# loss = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Evaluate model
pred = tf.argmax(y_pred, 1)
true = tf.argmax(Y, 1)
correct_prediction = tf.equal(pred, true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Training cycle
all_x_test = mnist.test.images
all_y_test = mnist.test.labels
start = time.time()
for epoch_i in range(training_epochs):
    ave_cost = 0
    for batch_i in range(total_batch):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, c = sess.run(
            [optimizer, loss],
            feed_dict={
                X: batch_x,
                Y: batch_y,
                keep_prob: train_dropout
            })
        ave_cost += c / total_batch
    # Display logs per epoch step
    if epoch_i % 1 == 0:
        train_acc = sess.run(accuracy,feed_dict={
            X: batch_x,
            Y: batch_y,
            keep_prob: train_dropout
        })
        test_acc = sess.run(accuracy,feed_dict={
            X: all_x_test,
            Y: all_y_test,
            keep_prob: test_dropout
        })
        print("Epoch:%2d Batch:%4d" % (epoch_i + 1, batch_i + 1),
              "train_acc=%.3f" % train_acc, "test_acc=%.3f" % test_acc,
              "train_cost=%5.3f" % ave_cost)
end = time.time()
print("Process Time :%.2f s" % (end - start))

# Calculate accuracy
acc = sess.run(
    accuracy,
    feed_dict={
        X: all_x_test,
        Y: all_y_test,
        keep_prob: test_dropout
    })
print("test accuracy=%.3f" % acc)
sess.close()

Epoch: 1 Batch: 550 train_acc=0.930 test_acc=0.954 train_cost=0.481
Epoch: 2 Batch: 550 train_acc=0.970 test_acc=0.966 train_cost=0.152
Epoch: 3 Batch: 550 train_acc=0.970 test_acc=0.968 train_cost=0.106
Epoch: 4 Batch: 550 train_acc=0.980 test_acc=0.983 train_cost=0.081
Epoch: 5 Batch: 550 train_acc=0.980 test_acc=0.979 train_cost=0.071
Epoch: 6 Batch: 550 train_acc=0.980 test_acc=0.982 train_cost=0.059
Epoch: 7 Batch: 550 train_acc=0.970 test_acc=0.984 train_cost=0.056
Epoch: 8 Batch: 550 train_acc=0.970 test_acc=0.985 train_cost=0.050
Epoch: 9 Batch: 550 train_acc=1.000 test_acc=0.987 train_cost=0.047
Epoch:10 Batch: 550 train_acc=0.970 test_acc=0.986 train_cost=0.042
Epoch:11 Batch: 550 train_acc=1.000 test_acc=0.985 train_cost=0.040
Epoch:12 Batch: 550 train_acc=1.000 test_acc=0.989 train_cost=0.035
Epoch:13 Batch: 550 train_acc=0.980 test_acc=0.988 train_cost=0.032
Epoch:14 Batch: 550 train_acc=0.980 test_acc=0.988 train_cost=0.032
Epoch:15 Batch: 550 train_acc=0.980 test_acc=0.9